# SSD Jet Detection Training

In [1]:
# Import GPU libs

import setGPU
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

setGPU: Setting GPU to: 0


In [2]:
# Other imports

import numpy as np
import simplejson as json

from math import ceil

In [3]:
# Set presentation settings

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as tick

from matplotlib import cm
from matplotlib.colors import SymLogNorm
from mpl_toolkits import mplot3d

%matplotlib inline

matplotlib.rcParams["figure.figsize"] = (16.0, 6.0)

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Anonymous Pro for Powerline']

matplotlib.rcParams["axes.spines.left"] = True
matplotlib.rcParams["axes.spines.top"] = True
matplotlib.rcParams["axes.spines.right"] = True
matplotlib.rcParams["axes.spines.bottom"] = True
matplotlib.rcParams["axes.labelsize"] = 16
matplotlib.rcParams["axes.titlesize"] = 14

matplotlib.rcParams["xtick.top"] = True
matplotlib.rcParams["ytick.right"] = True
matplotlib.rcParams["xtick.direction"] = "in"
matplotlib.rcParams["ytick.direction"] = "in"
matplotlib.rcParams["xtick.labelsize"] = 10
matplotlib.rcParams["ytick.labelsize"] = 10
matplotlib.rcParams["xtick.major.size"] = 10
matplotlib.rcParams["ytick.major.size"] = 10
matplotlib.rcParams["xtick.minor.size"] = 5
matplotlib.rcParams["ytick.minor.size"] = 5
matplotlib.rcParams["xtick.minor.visible"] = True

matplotlib.rcParams["lines.linewidth"] = 2

matplotlib.rcParams["legend.fontsize"] = 14

with open('../data/palette.json') as json_file:
    color_palette = json.load(json_file)

In [4]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [5]:
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.


In [6]:
from ssd.generator import DataGenerator
from ssd.keras_ssd7 import build_model
from ssd.keras_ssd_loss import SSDLoss
from ssd.ssd_input_encoder import SSDInputEncoder
from ssd.ssd_output_decoder import decode_detections

In [8]:
# Model configuration parameters

SAVE_PATH = '/data/adpol'
MODEL_NAME = 'ceva-cms-jet-ssd'
DATA_SOURCE = '/eos/user/a/adpol/ceva'
TRAINING_EPOCHS = 20
SPLIT = [0.1, 0.1, 0.1]
MAX_EVENTS = 100

classes = ['background', 'b', 'h', 'W', 't', 'q']

img_height = 452 # Pixel height
img_width = 340 # Pixel width
img_channels = 1 # Number of channels
n_classes = 4 # Number of target classes

# Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
intensity_mean = None 
intensity_range = None

# An explicit list of anchor box scaling factors. If this is passed, it will override `min_scale` and `max_scale`.
scales = [0.16, 0.4, 0.6, 0.8, 0.96]

# The list of aspect ratios for the anchor boxes
aspect_ratios = [1.0]
two_boxes_for_ar1 = True # Whether or not you want to generate two anchor boxes for aspect ratio 1
steps = None # In case you'd like to set the step sizes for the anchor box grids manually; not recommended
offsets = None # In case you'd like to set the offsets for the anchor box grids manually; not recommended
clip_boxes = True # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [1.0, 1.0, 1.0, 1.0] # The list of variances by which the encoded target coordinates are scaled
normalize_coords = True # Whether or not the model is supposed to use coordinates relative to the image size

In [9]:
# Compile the model

K.clear_session()

model = build_model(image_size=(img_height, img_width, img_channels),
                    n_classes=n_classes,
                    mode='training',
                    l2_regularization=0.0005,
                    scales=scales,
                    aspect_ratios_global=aspect_ratios,
                    aspect_ratios_per_layer=None,
                    two_boxes_for_ar1=two_boxes_for_ar1,
                    steps=steps,
                    offsets=offsets,
                    clip_boxes=clip_boxes,
                    variances=variances,
                    normalize_coords=normalize_coords,
                    subtract_mean=intensity_mean,
                    divide_by_stddev=intensity_range)

# Instantiate an Adam optimizer and the SSD loss function and compile the model
adam = Adam(lr=0.00001, beta_1=0.7, beta_2=0.9, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [10]:
# Training data generator

hdf5_dataset_paths_list = []

for i in np.arange(0, 10*SPLIT[0], dtype=np.int16):
    for j in ['bb', 'tt', 'WW', 'hh']:
        hdf5_dataset_paths_list.append('%s/RSGraviton_%s_NARROW_%s.h5' % (DATA_SOURCE, j, i))

train_dataset = DataGenerator(hdf5_dataset_paths=hdf5_dataset_paths_list, max_size=MAX_EVENTS)

In [11]:
# Training data generator

hdf5_dataset_paths_list = []

for i in np.arange(10*SPLIT[0], 10*(SPLIT[0]+SPLIT[1]), dtype=np.int16):
    for j in ['bb', 'tt', 'WW', 'hh']:
        hdf5_dataset_paths_list.append('%s/RSGraviton_%s_NARROW_%s.h5' % (DATA_SOURCE, j, i))

val_dataset = DataGenerator(hdf5_dataset_paths=hdf5_dataset_paths_list, max_size=MAX_EVENTS)

In [12]:
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   400
Number of images in the validation dataset:	   400


In [13]:
# Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

batch_size = 100

predictor_sizes = [model.get_layer('classes4').output_shape[1:3],
                   model.get_layer('classes5').output_shape[1:3],
                   model.get_layer('classes6').output_shape[1:3],
                   model.get_layer('classes7').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_global=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.3,
                                    normalize_coords=normalize_coords)

In [14]:
# Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'})

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=True,
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'})

## Model training

In [15]:
# Check if GPU is available

print('GPU Available? %s' % (len(K.tensorflow_backend._get_available_gpus()) > 0))

GPU Available? True


In [16]:
# Define callbacks

model_checkpoint = ModelCheckpoint(filepath='%s/%s.h5' % (SAVE_PATH, MODEL_NAME),
                                   monitor='val_loss',
                                   verbose=0,
                                   save_best_only=True,
                                   save_weights_only=True,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename='%s/%s.csv' % (SAVE_PATH, MODEL_NAME),
                       separator=',',
                       append=False)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=10,
                               verbose=0)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=8,
                                         verbose=0,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)

callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
             reduce_learning_rate]

In [17]:
history = model.fit_generator(generator=train_generator,
                              use_multiprocessing=False,
                              validation_data=train_generator,
                              steps_per_epoch=int(np.floor(train_dataset_size/batch_size)),
                              validation_steps=int(np.floor(train_dataset_size/batch_size)),
                              epochs=TRAINING_EPOCHS,
                              workers=0,
                              callbacks=callbacks)

Epoch 1/20
4/4 [==============================] - 194s 49s/step - loss: 33.6100 - val_loss: 33.2547
Epoch 2/20
4/4 [==============================] - 12s 3s/step - loss: 33.1258 - val_loss: 32.6958
Epoch 3/20
4/4 [==============================] - 11s 3s/step - loss: 32.6375 - val_loss: 32.1571
Epoch 4/20
4/4 [==============================] - 11s 3s/step - loss: 32.1222 - val_loss: 31.6266
Epoch 5/20
4/4 [==============================] - 12s 3s/step - loss: 31.6797 - val_loss: 31.1199
Epoch 6/20
4/4 [==============================] - 12s 3s/step - loss: 31.1870 - val_loss: 30.6051
Epoch 7/20
4/4 [==============================] - 12s 3s/step - loss: 30.7139 - val_loss: 30.1445
Epoch 8/20
4/4 [==============================] - 11s 3s/step - loss: 30.2466 - val_loss: 29.6317
Epoch 9/20
4/4 [==============================] - 11s 3s/step - loss: 29.7824 - val_loss: 29.1578
Epoch 10/20
4/4 [==============================] - 11s 3s/step - loss: 29.3267 - val_loss: 28.6717
Epoch 11/20
4/4 [